<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parte 1 - Solicitação de dados primários ao usuário

In [17]:
# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO #

# Pedir o nome da cidade e o total de resíduos produzidos por kg/mês na cidade
cidade = input('Informe o nome da cidade: ')
residuos_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))

# Pedir ao usuário a quantidade de bairros
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

bairros = []  # Lista para armazenar os nomes dos bairros
habitantes_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Pedir ao usuário os nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    bairros.append(bairro)

# Pedir ao usuário o número de habitantes de cada bairro e validar se é um número inteiro válido
for bairro in bairros:
    while True:
        try:
            habitantes = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
            if habitantes < 0:
                print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
                continue
            else:
                habitantes_bairro.append(habitantes)
                break
        except ValueError:
            print('Por favor, insira um número inteiro para o número de habitantes.')

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA #

total_habitantes = sum(habitantes_bairro)

residuos_por_bairro_semanal = []

for i, habitantes in enumerate(habitantes_bairro):
    proporcao = habitantes / total_habitantes
    residuos_bairro_semanal = round((proporcao * residuos_cidade) / 4)  # Dividindo por 4 para obter a quantidade semanal
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

# PARTE 3 - DIMENSIONAMENTO DOS DEPÓSITOS, SETOR DE TRIAGEM E ESTAÇÃO DE COLETA POR BAIRRO #

# Fator de conversão: cada unidade de resíduo ocupa 5 cm², convertendo para metros quadrados (0.05 m²)
fator_area_armaz_por_kg = 0.05

# Listas para armazenar os resultados do dimensionamento por bairro
alturas_armaz_por_bairro = []
larguras_armaz_por_bairro = []
areas_reais_armaz_por_bairro = []
perimetros_armaz_por_bairro = []

# Dados de dimensionamento do setor de triagem por faixa de geração de resíduos diários
def calcular_dimensoes_setor_triagem(residuos_diarios):
    if residuos_diarios <= 0.25 * 7:  # Transformando de semanal para diário
        return {'area': 55, 'comprimento': 0.0}
    elif 0.25 * 7 < residuos_diarios <= 0.6 * 7:
        return {'area': 80, 'comprimento': 0.0}
    elif 0.6 * 7 < residuos_diarios <= 1 * 7:
        return {'area': 180, 'comprimento': 0.0}
    elif residuos_diarios > 1 * 7:
        return {'area': 400, 'comprimento': 0.0}
    else:
        area_media = (55 + 75) / 2  # Média entre as áreas mínima e máxima
        return {'area': area_media, 'comprimento': 0.0}

# Lista para armazenar o número de estações de coleta por bairro
num_estacoes_por_bairro = []

# Limite máximo de área para a Estação de Coleta por bairro
area_maxima_estacao = 1500.0  # 1500 metros quadrados

# Altura mínima definida para depósitos e setor de triagem (em metros)
altura_minima = 4.0  # 4 metros

# Itera sobre cada bairro para calcular o dimensionamento dos depósitos, setor de triagem e estação de coleta
for bairro, residuos_diarios in zip(bairros, residuos_por_bairro_semanal):
    # Calcula a área necessário para o armazenamento de resíduos para as estações por bairro com base na quantidade de resíduos semanais
    area_necessaria_armaz_bairro = residuos_diarios * fator_area_armaz_por_kg

    # Dimensionamento do depósito (retângulo) do bairro atual
    altura_armaz = altura_minima
    largura_armaz = area_necessaria_armaz_bairro / altura_armaz
    area_real_armaz = largura_armaz * altura_armaz

    # Ajusta as dimensões do retângulo até alcançar a área necessária para o bairro
    while area_real_armaz < area_necessaria_armaz_bairro:
        altura_armaz += 1
        largura_armaz = area_necessaria_armaz_bairro / altura_armaz
        area_real_armaz = largura_armaz * altura_armaz

    # Calcula o perímetro do depósito (retângulo) do bairro
    perimetro_armaz = 2 * (largura_armaz + altura_armaz)

    # Calcula as dimensões do setor de triagem
    dimensoes_triagem = calcular_dimensoes_setor_triagem(residuos_diarios)
    area_triagem = dimensoes_triagem['area']
    comprimento_triagem = dimensoes_triagem['comprimento']
    largura_triagem = 2 * largura_armaz  # Largura do setor de triagem é duas vezes a largura do depósito

    # Calcula a área total da Estação de Coleta
    area_total_estacao = 2 * area_real_armaz + area_triagem

    # Verifica quantas estações de coleta são necessárias para não ultrapassar o limite máximo de área
    num_estacoes = max(1, area_total_estacao / area_maxima_estacao)  # Garante que haja pelo menos 1 estação de coleta
    num_estacoes_por_bairro.append(num_estacoes)

    # Armazena os resultados para o bairro atual nas listas correspondentes
    alturas_armaz_por_bairro.append(altura_armaz)
    larguras_armaz_por_bairro.append(largura_armaz)
    areas_reais_armaz_por_bairro.append(area_real_armaz)
    perimetros_armaz_por_bairro.append(perimetro_armaz)

# PARTE 4 - IMPRESSÃO DOS RESULTADOS POR BAIRRO #

print(f'CIDADE: {cidade}')
print('=-=' * 30)
print()

# Impressão da quantidade de resíduos gerados por semana em cada bairro
print('QUANTIDADE DE RESÍDUOS GERADOS POR SEMANA POR BAIRRO')
print()
for bairro, residuos_semanais in zip(bairros, residuos_por_bairro_semanal):
    print(f'No bairro {bairro}, a quantidade de resíduos por semana é de aproximadamente {residuos_semanais} kg.')
    print()

print('=-=' * 30)
print()

# Impressão do dimensionamento dos depósitos, setor de triagem e estação de coleta por bairro
print('DIMENSIONAMENTO DOS DEPÓSITOS, SETOR DE TRIAGEM E ESTAÇÃO DE COLETA POR BAIRRO')
print()
for bairro, altura_armaz, largura_armaz, area_real_armaz, perimetro_armaz, num_estacoes in zip(bairros, alturas_armaz_por_bairro, larguras_armaz_por_bairro, areas_reais_armaz_por_bairro, perimetros_armaz_por_bairro, num_estacoes_por_bairro):
    print(f'Bairro: {bairro}')
    print(f'Altura do depósito de armazenamento: {altura_armaz:.2f} metros')
    print(f'Largura do depósito de armazenamento: {largura_armaz:.2f} metros')
    print(f'Área real do depósito de armazenamento: {area_real_armaz:.2f} metros quadrados')
    print(f'Perímetro do depósito de armazenamento: {perimetro_armaz:.2f} metros')
    print(f'Número mínimo de Estações de Coleta necessárias: {num_estacoes:.0f}')
    print()

print('=-=' * 30)


Informe o nome da cidade: Natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 221.200
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: Lagoa
Digite o nome do 2º bairro: Lagoinha
Digite o número de habitantes do bairro Lagoa: 4000
Digite o número de habitantes do bairro Lagoinha: 3000
CIDADE: Natal
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

QUANTIDADE DE RESÍDUOS GERADOS POR SEMANA POR BAIRRO

No bairro Lagoa, a quantidade de resíduos por semana é de aproximadamente 32 kg.

No bairro Lagoinha, a quantidade de resíduos por semana é de aproximadamente 24 kg.

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

DIMENSIONAMENTO DOS DEPÓSITOS, SETOR DE TRIAGEM E ESTAÇÃO DE COLETA POR BAIRRO

Bairro: Lagoa
Altura do depósito de armazenamento: 4.00 metros
Largura do depósito de armazenamento: 0.40 metros
Área real do depósito de armazenamento: 1.60 metros

In [ ]:

#_____________________________________________________________________________________________________________________________________________________________________________

# PARTE 6 - QUANTOS CAMINHÕES SERÃO NECESSÁRIOS #                                                                                      (priorizar estruturas de controle)

import math

caminhão = 7000
Quantidade_de_caminhões_necessarios = math.ceil(residuos_semanais/caminhão)
print(f"Quantidade de caminhões necessários: {Quantidade_de_caminhões_necessarios}")

